In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import  make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [3]:
df=pd.read_csv('RTA_preprocessed.csv')
df.head()

,Age_band_of_driver,Sex_of_driver,Educational_level,Vehicle_driver_relation,Driving_experience,Type_of_vehicle,Area_accident_occured,Lanes_or_Medians,Types_of_Junction,Road_surface_type,Light_conditions,Weather_conditions,Type_of_collision,Vehicle_movement,Pedestrian_movement,Cause_of_accident,Accident_severity,Number_of_vehicles_involved
0,18-30,male,junior high school,employee,5-10yr,bus,office areas,other,y shape,earth roads,darkness - lights lit,normal,vehicle with vehicle collision,going straight,not a pedestrian,changing lane to the right,Slight Injury,2
1,18-30,male,junior high school,employee,above 10yr,lorry,industrial areas,other,y shape,earth roads,daylight,normal,collision with roadside-parked vehicles,going straight,crossing from driver's nearside,changing lane to the right,Slight Injury,2
2,18-30,male,junior high school,employee,1-2yr,car,residential areas,undivided two way,y shape,asphalt roads,daylight,normal,collision with roadside-parked vehicles,u-turn,not a pedestrian,moving backward,Serious Injury,2
3,31-50,male,above high school,employee,no licence,car,office areas,undivided two way,no junction,earth roads,daylight,normal,collision with animals,going straight,not a pedestrian,no priority to pedestrian,Serious Injury,2
4,31-50,male,junior high school,employee,above 10yr,car,office areas,undivided two way,no junction,asphalt roads,daylight,normal,collision with animals,going straight,not a pedestrian,no distancing,Serious Injury,2


In [4]:
x=df.drop('Accident_severity',axis=1)
y=df['Accident_severity']

In [5]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(y_train.groupby(y_train).count())

Accident_severity
Fatal injury        61
Serious Injury     597
Slight Injury     3471
Name: Accident_severity, dtype: int64


In [6]:
ohe=OneHotEncoder()
ohe.fit(x[['Age_band_of_driver','Sex_of_driver','Educational_level','Vehicle_driver_relation','Driving_experience','Type_of_vehicle','Area_accident_occured','Lanes_or_Medians','Types_of_Junction','Road_surface_type','Light_conditions','Weather_conditions','Type_of_collision','Vehicle_movement','Pedestrian_movement','Cause_of_accident']])

OneHotEncoder()

In [7]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Age_band_of_driver','Sex_of_driver','Educational_level','Vehicle_driver_relation','Driving_experience','Type_of_vehicle','Area_accident_occured','Lanes_or_Medians','Types_of_Junction','Road_surface_type','Light_conditions','Weather_conditions','Type_of_collision','Vehicle_movement','Pedestrian_movement','Cause_of_accident']),remainder='passthrough')

In [8]:
rf=RandomForestClassifier(n_estimators=100,random_state=42)
pipe=make_pipeline(column_trans,rf)
pipe.fit(X_train,y_train)
pipe.score(X_test,y_test)
print(classification_report(y_test,pipe.predict(X_test)))
print(accuracy_score(y_test,pipe.predict(X_test)))

                precision    recall  f1-score   support

  Fatal injury       0.00      0.00      0.00        13
Serious Injury       0.25      0.01      0.01       137
 Slight Injury       0.86      1.00      0.92       883

      accuracy                           0.85      1033
     macro avg       0.37      0.33      0.31      1033
  weighted avg       0.76      0.85      0.79      1033

0.8528557599225557


c:\Users\MUFEEZ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\MUFEEZ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\MUFEEZ\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
# Saving the model to disk
with open('rf2_model.pkl', 'wb') as f:
    pickle.dump(pipe, f)